## MetaCost



In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from metacost import MetaCost

In [2]:
# load data
# only a few observations to speed the computaton

data = pd.read_csv('../kdd2004.csv').sample(10000)

# remap target class to 0 and 1
data['target'] = data['target'].map({-1:0, 1:1})

data.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,target
7888,52.66,24.47,0.16,10.5,-30.5,3067.9,-0.43,-0.04,15.5,-107.0,...,2582.9,-0.80,0.93,2.0,-138.0,2478.1,-1.38,0.07,0.47,0
32228,79.05,22.00,0.53,-4.0,-16.0,2033.0,1.59,-0.14,-1.5,-111.5,...,1475.2,0.09,2.43,24.0,-191.0,729.2,1.09,0.25,0.64,0
15430,52.38,36.07,-0.76,-14.0,11.0,711.6,1.38,1.22,10.5,-67.5,...,1322.1,-1.22,-0.24,-1.0,-23.0,237.4,-0.15,0.29,-0.20,0
51691,67.13,22.30,1.43,-10.5,69.0,2373.4,2.26,2.09,5.0,-80.5,...,3419.3,0.73,-0.59,13.0,-76.0,333.2,1.23,0.33,0.58,0
56544,93.08,27.27,2.07,44.5,-43.0,1230.3,0.18,0.95,20.5,-70.0,...,1071.8,0.37,-0.67,1.0,-63.0,702.6,0.47,0.53,0.07,0


In [3]:
# imbalanced target

data.target.value_counts() / len(data)

0    0.9897
1    0.0103
Name: target, dtype: float64

In [4]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),  # drop the target
    data['target'],  # just the target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((7000, 74), (3000, 74))

In [5]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
25540,25.89,31.03,0.59,22.5,-5.0,734.6,1.02,-0.55,4.0,-51.0,...,-16.0,1122.9,-0.83,-0.51,-2.0,-44.0,473.8,0.18,0.48,0.27
138573,55.20,31.08,1.07,-19.0,26.0,1941.8,0.15,0.80,7.5,-70.5,...,11.0,1297.1,1.17,1.22,12.0,-67.0,84.1,2.10,0.25,0.24
36887,35.06,36.07,0.30,18.0,46.5,1788.7,0.43,-0.87,-16.5,-57.5,...,55.0,2234.9,-0.26,0.42,1.0,-44.0,212.7,0.32,0.31,0.10
107199,17.74,32.00,-1.52,1.0,28.0,787.2,0.99,-1.91,-37.0,-38.5,...,4.0,635.0,0.81,0.63,3.0,-34.0,450.5,-0.68,0.81,0.57
257,89.00,23.60,0.81,7.5,-13.0,678.5,1.03,0.65,4.5,-64.0,...,-7.0,1192.2,-1.08,3.83,5.0,-50.0,223.1,0.11,0.42,0.52


## Set up Logistic regression

In [6]:
# set up the estimator we would like to ensemble

logit = LogisticRegression(
    penalty='l2',
    solver='newton-cg',
    random_state=0,
    max_iter=10,
    n_jobs=4,
)

## MetaCost

With no cost

In [7]:
cost_matrix = np.array([[0, 1], [1, 0]])
cost_matrix

array([[0, 1],
       [1, 0]])

In [8]:
metacost_ = MetaCost(estimator=logit,
                     cost_matrix=cost_matrix,
                     n_estimators=50,
                     n_samples=None,
                     p=True,
                     q=True)

In [9]:
metacost_.fit(X_train, y_train)

resampling data and training ensemble
Finished training ensemble
evaluating optimal class per observation
Finished re-assigning labels
Training model on new data
Finished training model on data with new labels


In [10]:
metacost_.predict_proba(X_train)

array([[9.99791710e-01, 2.08290383e-04],
       [9.99999974e-01, 2.58940074e-08],
       [9.99999983e-01, 1.73612076e-08],
       ...,
       [9.99997623e-01, 2.37692304e-06],
       [9.99978408e-01, 2.15924268e-05],
       [9.99992548e-01, 7.45181288e-06]])

In [11]:
print('Train set')
pred = metacost_.predict_proba(X_train)
print(
    'MetaCost roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

print('Test set')
pred = metacost_.predict_proba(X_test)
print(
    'MetaCost roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

Train set
MetaCost roc-auc: 0.8841745753043067
Test set
MetaCost roc-auc: 0.8207058823529412


## MetaCost

With costs

TN | FN
 
FP | TP

In [12]:
cost_matrix = np.array([[0, 100], [1, 0]])
cost_matrix

array([[  0, 100],
       [  1,   0]])

In [13]:
metacost2 = MetaCost(estimator=logit,
                     cost_matrix=cost_matrix,
                     n_estimators=50,
                     n_samples=None,
                     p=True,
                     q=True)

In [14]:
metacost2.fit(X_train, y_train)

print('Train set')
pred = metacost2.predict_proba(X_train)
print(
    'MetaCost roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

print('Test set')
pred = metacost2.predict_proba(X_test)
print(
    'MetaCost roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

resampling data and training ensemble
Finished training ensemble
evaluating optimal class per observation
Finished re-assigning labels
Training model on new data
Finished training model on data with new labels
Train set
MetaCost roc-auc: 0.9220526896776534
Test set
MetaCost roc-auc: 0.8246319327731092


In [15]:
y_train.reset_index(drop=True)

0       0
1       0
2       0
3       0
4       0
       ..
6995    0
6996    0
6997    0
6998    0
6999    0
Name: target, Length: 7000, dtype: int64

In [16]:
tmp = pd.concat([metacost2.y_, y_train.reset_index(drop=True)], axis=1)

tmp.head()

,0,target
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0


In [17]:
tmp[tmp[0]!=tmp['target']][['target', 0]]

,target,0
4,0,1
5,0,1
13,0,1
21,0,1
22,0,1
...,...,...
6965,0,1
6983,0,1
6985,0,1
6989,0,1


In theory, we should only be re-labeling observations from class 0 to class 1, but in practice that does not happen.

In [18]:
np.sum( np.where(metacost2.y_ != y_train.reset_index(drop=True),1,0) )

1444

In [19]:
np.sum( np.where(metacost2.y_ == y_train.reset_index(drop=True),1,0) )

5556

## Conclusion

We can wrap a model to make it cost-sensitive utilizing metacost.

### Important

The code I provided, does not allow reproducible results, because at the moment the class MetaCost does not incorporate a seed when re-sampling the data.

MetaCost might be incorporated to Sklearn, there is a PR open:
https://github.com/scikit-learn/scikit-learn/pull/16525 

**HOMEWORK**

Go ahead and compare how many observations are relabeled if we apply no extra cost to the minority class, or different costs to 100.

Also insteresting, compare the performance of bagging with cost_sensitive learning (adding the parameter class_weight) with MetaCost.